In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import os

os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

spark = SparkSession.builder.appName('ABC').getOrCreate();
df = spark.read.csv("taxis.csv", header = True);
df2 = spark.read.csv("zone.csv", header = True);
df.printSchema();

df2.printSchema();

AnalysisException: u'Path does not exist: file:/home/droro/Documents/Python Proyectos/BigData-Scripts/Taller Ventanas/taxis.csv;'

## Punto 1 - Distancia Promedio de un recorrido en Taxi NYC

In [23]:

dfD = df.select(f.col('trip_distance').cast('double')).groupBy().avg().show();

+------------------+
|avg(trip_distance)|
+------------------+
|3.0785054986122593|
+------------------+



## Punto 2  - Cuántas formas de pago hay?

In [34]:
dfP = df.select('payment_type').dropDuplicates().groupBy().count().show();

+-----+
|count|
+-----+
|    4|
+-----+



## Punto 3 - Taxi Con mayor Numero de Viajes

In [65]:
dfT = df.select('VendorID').groupBy('VendorID').count().orderBy(f.desc('count'));
dfT2 = dfT.select('VendorID').limit(1).show();

+--------+
|VendorID|
+--------+
|       2|
+--------+



## Punto 4 - Listar número de viajes por dia en el mes de junio de 2019

In [18]:
dfL = df.select(f.split('tpep_pickup_datetime','-')[0].alias('Year'),f.split('tpep_pickup_datetime','-')[1].alias('Month'),f.split(f.split('tpep_pickup_datetime','-')[2],' ')[0].alias('day'));
dfL2 = dfL.where('Year = 2019 and Month = 6').groupBy('day').count().sort('day').show(30);

+---+------+
|day| count|
+---+------+
| 01|244836|
| 02|209385|
| 03|227670|
| 04|227926|
| 05|253309|
| 06|261778|
| 07|250221|
| 08|229460|
| 09|190194|
| 10|221211|
| 11|239921|
| 12|251902|
| 13|262813|
| 14|242520|
| 15|221153|
| 16|187956|
| 17|215627|
| 18|242304|
| 19|246680|
| 20|255073|
| 21|248782|
| 22|221070|
| 23|200305|
| 24|218503|
| 25|241479|
| 26|249641|
| 27|253765|
| 28|242109|
| 29|216025|
| 30|166871|
+---+------+



## Punto 5 - Área donde se recoge mayor número de pasajeros 

In [17]:
dfz = df.select(f.col('PULocationID').alias('LocationID')).groupBy('LocationID').count();
dfz2 = dfz.orderBy(f.desc('count')).limit(1);
dfzTemp = df2.select('LocationID','Zone').dropDuplicates();
dfz3 = dfz2.join(dfzTemp,dfzTemp.LocationID == dfz2.LocationID ).show()

+----------+------+----------+--------------------+
|LocationID| count|LocationID|                Zone|
+----------+------+----------+--------------------+
|       237|295057|       237|Upper East Side S...|
+----------+------+----------+--------------------+



## Punto 6 - Número de viajes que se dirigieron al “Bronx”

In [2]:
dfbr = df2.where("Borough = 'Bronx' ").select('LocationID');
dfbr2 = df.select('DOLocationID').join(dfbr, dfbr.LocationID == df.DOLocationID);
dfbr3 = dfbr2.select('LocationID').groupBy().count().show();

+-----+
|count|
+-----+
|48267|
+-----+



## Punto 7 - Número promedio de personas por viaje que se dirigen al aeropuerto JFK


In [20]:
df_jfk2 = df2.where(" Zone = 'JFK Airport'").select('LocationID');
df_jfk1 = df.select(f.col('DOLocationID'),f.col('passenger_count').cast('double'));

df_jfk3 = df_jfk2.join(df_jfk1, df_jfk2.LocationID == df_jfk1.DOLocationID).groupBy('LocationID').avg('passenger_count').show();

+----------+--------------------+
|LocationID|avg(passenger_count)|
+----------+--------------------+
|       132|  1.6181272818577885|
+----------+--------------------+



## Punto 8 - Distancia y Costo promedio de tomar un taxi del Aeropuerto JFK a Manhattan Valley